In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [2]:
import gc
import requests
from collections import Counter

import pandas as pd
from tqdm import tqdm
from numpy import ndarray
from datasets import Dataset, DatasetDict

tqdm.pandas()

/home/harddisk/projects/legal_ml/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path_files = "./data/interim/split_9111_dataset"

res = []
for name_file in tqdm(os.listdir(path_files)):
    res.append(pd.read_parquet(os.path.join(path_files, name_file)))
    break

df = pd.concat(res)

del res
gc.collect()

  0%|          | 0/13 [00:14<?, ?it/s]


0

In [29]:
# запись из-за который возникала ошибка
df_bad_record = pd.DataFrame(
    {
        "id": 18536837,
        "title": "Как мне получить субсидии на ком. платежи, если я ухаживаю за пожилым человеком старше 80 лет.",
        "description": "Как мне получить субсидии на ком. платежи, если я ухаживаю за пожилым человеком старше 80 лет.",
        "answers": [
            [
                {
                    "user_name": "Парфенов В.Н.",
                    "status": "Юрист",
                    "rating": 4.6,
                    "text": "К сожалению, то что вы ухаживаете за пожилым человеком старше 80 лет.-не является основанием для получения субсидии на оплату коммунальных услуг стст 153-155 ЖК РФ.",
                }
            ]
        ],
    }
)
df = pd.concat([df, df_bad_record])

In [30]:
df.head(5)

,id,title,description,answers
0,16420931,Необходимость работы сверх установленного граф...,"Я работы в мосметро в ПТБ, и график у меня сме...","[{'user_name': 'Чередниченко В.А.', 'status': ..."
1,16420932,"Хочу сестру лишить родительских прав, но не хо...","Хочу сестру лешить родительских прав, но не х...","[{'user_name': 'Тома А. В.', 'status': 'Юрист'..."
2,16420933,Можно ли при вступлении в наследство у нотариу...,Можно ли при вступлении в наследство у нотари...,"[{'user_name': 'Елисеева Е. Е.', 'status': 'Юр..."
3,16420940,Вопрос о льготах на оплату детского сада для д...,У меня такой вопрос у меня 2 детей. У меня был...,"[{'user_name': 'Ростовцев В. В.', 'status': 'Ю..."
4,16420941,Будут ли приходить штрафы с камер видео-фото н...,Если выписали протокол за тонировку на машину...,"[{'user_name': 'Попов П. Е.', 'status': 'Юрист..."


In [31]:
print(df["description"].sample().to_list()[0])

Может ли адвокат подать исковое заявление с предоставлением копии, а предоставить для сверки оригиналы  после подачи иска.


In [32]:
df["description"].iloc[0]

'Я работы в мосметро в ПТБ, и график у меня сменный 12 часов! В месяц выходит 186 часов и они не вмещаются в во все рабочие дни и ставят нам не 12 часов а 13 часов а если не хочешь дорабатывать, выходи в выходной день! Скажите, кто прав и законно ли все это? Потому как в трудовом договоре  честно написано-день 12, ночь 12, отсыпной и выходной! Спасибо.'

### Анализ датасета

In [33]:
print(f"Всего вопросов: {df.shape[0]:,}")

Всего вопросов: 11,292,803


Собираем статистику по вопросам и ответам:
- сколько и кто отвечал
- распределение рейтинга


In [34]:
counter_expert = []
counter_rating = []
counter_len_chat = []
author_in_chat = []


def stat(answers):
    answers = list(answers)
    len_answers = len(answers)

    counter_len_chat.append(len_answers)
    try:
        counter_expert.extend([i["status"] for i in answers])
        counter_rating.extend([i["rating"] for i in answers])
        for i in answers:
            if i["status"] == "Автор вопроса":
                author_in_chat.append(True)
                break
        else:
            author_in_chat.append(False)
    except:
        print(answers)

In [35]:
for i in tqdm(df["answers"].to_list()):
    stat(i)

100%|██████████| 11292803/11292803 [00:33<00:00, 333173.05it/s]


In [36]:
counter_len_chat_series = pd.Series(
    [i if i in [1, 2, 3, 4, 5] else -1 for i in counter_len_chat]
)
author_in_chat_series = pd.Series(author_in_chat)

counter_len_chat_series.value_counts(normalize=True)

 1    0.581362
 2    0.214937
 3    0.094587
 4    0.044428
-1    0.040818
 5    0.023868
Name: proportion, dtype: float64

In [37]:
author_in_chat_series.value_counts(normalize=True)

False    0.916276
True     0.083724
Name: proportion, dtype: float64

In [38]:
def norm(answers, description):
    answers = list(answers)
    try:
        len_answ = len(answers)
        rating = answers[0]["rating"]
        text = answers[0]["text"]
        len_description = len(description)
        len_text = len(text)
    except:
        len_answ, rating, text, len_description, len_text = None, None, None, None, None
    return len_answ, rating, text, len_description, len_text

In [39]:
len(author_in_chat)

11292803

In [40]:
df["author_in_chat"] = author_in_chat
df["counter_len_chat"] = counter_len_chat
df["counter_len_chat_bins"] = counter_len_chat_series

In [41]:
df.head()

,id,title,description,answers,author_in_chat,counter_len_chat,counter_len_chat_bins
0,16420931,Необходимость работы сверх установленного граф...,"Я работы в мосметро в ПТБ, и график у меня сме...","[{'user_name': 'Чередниченко В.А.', 'status': ...",False,1,1
1,16420932,"Хочу сестру лишить родительских прав, но не хо...","Хочу сестру лешить родительских прав, но не х...","[{'user_name': 'Тома А. В.', 'status': 'Юрист'...",False,1,1
2,16420933,Можно ли при вступлении в наследство у нотариу...,Можно ли при вступлении в наследство у нотари...,"[{'user_name': 'Елисеева Е. Е.', 'status': 'Юр...",False,2,2
3,16420940,Вопрос о льготах на оплату детского сада для д...,У меня такой вопрос у меня 2 детей. У меня был...,"[{'user_name': 'Ростовцев В. В.', 'status': 'Ю...",False,3,3
4,16420941,Будут ли приходить штрафы с камер видео-фото н...,Если выписали протокол за тонировку на машину...,"[{'user_name': 'Попов П. Е.', 'status': 'Юрист...",True,2,2


In [42]:
df_short = df[(~df["author_in_chat"]) & (df["counter_len_chat"] < 6)]

In [43]:
df.shape[0] - df_short.shape[0]

1153658

In [44]:
df_short.shape

(10139145, 7)

In [45]:
df_short["answers"].iloc[0][0]

{'user_name': 'Чередниченко В.А.',
 'status': 'Адвокат',
 'rating': 3.8,
 'text': 'Полагаю, что незаконно, но для более конкретного ответа нужно смотреть ваши локальные внутренние документы организации, а также ваш трудовой договор. Как вариант, вы можете обратиться с жалобой в трудовую инспекцию, чтобы провела документальную проверку. Но это конфликт с работодателем, так что нужно быть готовым к любым последствиям.'}

In [46]:
df_short["answers"].iloc[11]

array([{'user_name': 'Иванова Е. А.', 'status': 'Юрист', 'rating': 0.0, 'text': 'Бухгалтерия учреждения алиментные выплаты осуществляет на основании исполнительного документа. Поучить исполнительный документ можно двумя способами.Составить алиментное соглашение. Либо через суд.'},
       {'user_name': 'Мерный М. А.', 'status': 'Юрист', 'rating': 1.8, 'text': 'Вы здесь не пишите ситуациютполностью и коллеги не могут ответить. Я вам всё объяснил. Вы время теряете драгоценное. Надо подавать.'}],
      dtype=object)

In [47]:
def func(answers):
    max_index = 0
    max_rating = 0
    for index, i in enumerate(answers):
        if max_rating < i["rating"]:
            max_index = index
            max_rating = i["rating"]
    return answers[max_index], max_rating


df_short[["answers_max", "max_rating"]] = df_short.apply(
    lambda x: func(x["answers"]), result_type="expand", axis=1
)

/tmp/ipykernel_497789/4077437054.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short[["answers_max", "max_rating"]] = df_short.apply(
/tmp/ipykernel_497789/4077437054.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short[["answers_max", "max_rating"]] = df_short.apply(


In [50]:
df_short_copy = df_short

In [51]:
df_short.head(3)

,id,title,description,answers,author_in_chat,counter_len_chat,counter_len_chat_bins,answers_max,max_rating
0,16420931,Необходимость работы сверх установленного граф...,"Я работы в мосметро в ПТБ, и график у меня сме...","[{'user_name': 'Чередниченко В.А.', 'status': ...",False,1,1,"{'user_name': 'Чередниченко В.А.', 'status': '...",3.8
1,16420932,"Хочу сестру лишить родительских прав, но не хо...","Хочу сестру лешить родительских прав, но не х...","[{'user_name': 'Тома А. В.', 'status': 'Юрист'...",False,1,1,"{'user_name': 'Тома А. В.', 'status': 'Юрист',...",0.0
2,16420933,Можно ли при вступлении в наследство у нотариу...,Можно ли при вступлении в наследство у нотари...,"[{'user_name': 'Елисеева Е. Е.', 'status': 'Юр...",False,2,2,"{'user_name': 'Гудкова Г.В.', 'status': 'Адвок...",1.5


In [58]:
df_short = df_short_copy[["description", "answers_max", "id"]]

In [59]:
URL = "http://127.0.0.1:8080/embed"


def emb(texts):
    return requests.post(URL, json={"inputs": texts}).json()


len(emb(["test1", "test2"])), len(emb(["test1", "test2"])[0])

(2, 1024)

In [1]:
from qdrant_client import QdrantClient, models

client = QdrantClient(url="http://localhost:6333")

COLLECTION_NAME = "questions"

In [2]:
client.delete_collection(collection_name=COLLECTION_NAME)

True

In [62]:
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=len(emb(["test1", "test2"])[0]), distance=models.Distance.COSINE
    ),
)

True

In [63]:
index0 = 0
len_df = len(df_short)
batch_size = 32
for index1 in tqdm(range(batch_size, len_df + batch_size, batch_size)):
    batch = df_short.iloc[index0:index1].to_dict(orient="records")
    index0 = index1

    batch = [
        {
            "id": i["id"],
            "question": i["description"],
            "answer": i["answers_max"]["text"],
            "rating": i["answers_max"]["rating"],
        }
        for i in batch
    ]

    embeddings_question = emb([i["question"] for i in batch])
    batch_points = [
        models.PointStruct(
            id=record["id"],
            payload={
                "question": record["question"],
                "answer": record["answer"],
                "rating": record["rating"],
            },
            vector=emb,
        )
        for emb, record in zip(embeddings_question, batch)
    ]

    client.upsert(
        collection_name=COLLECTION_NAME,
        points=batch_points,
    )

  0%|          | 108/316849 [00:11<9:38:42,  9.12it/s]


KeyboardInterrupt: 